In [1]:
pip install tensorflow==2.15.0


  Using cached tensorflow-2.16.1-cp39-cp39-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp39-cp39-win_amd64.whl.metadata (5.0 kB)
  Using cached keras-3.3.3-py3-none-any.whl.metadata (5.7 kB)
  Using cached rich-13.7.1-py3-none-any.whl.metadata (18 kB)
Using cached tensorflow-2.16.1-cp39-cp39-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.16.1-cp39-cp39-win_amd64.whl (376.9 MB)
Using cached keras-3.3.3-py3-none-any.whl (1.1 MB)
Using cached rich-13.7.1-py3-none-any.whl (240 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\arsen\\Healthylicious\\.venv\\Lib\\site-packages\\keras\\src\\backend\\torch\\__init__.py'
Check the permissions.



  Using cached tensorflow_recommenders-0.7.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tensorflow-2.16.1-cp39-cp39-win_amd64.whl.metadata (3.5 kB)
  Using cached tensorflow_intel-2.16.1-cp39-cp39-win_amd64.whl.metadata (5.0 kB)
Using cached tensorflow_recommenders-0.7.3-py3-none-any.whl (96 kB)
Using cached tensorflow-2.16.1-cp39-cp39-win_amd64.whl (2.1 kB)
Using cached tensorflow_intel-2.16.1-cp39-cp39-win_amd64.whl (376.9 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\Users\\arsen\\Healthylicious\\.venv\\Lib\\site-packages\\tensorflow\\compiler\\mlir\\stablehlo\\stablehlo_extension.pyd'
Check the permissions.



In [3]:
%pip install tensorflow_recommenders

  Using cached tensorflow_recommenders-0.7.3-py3-none-any.whl.metadata (4.6 kB)
Using cached tensorflow_recommenders-0.7.3-py3-none-any.whl (96 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import numpy as np
import pickle

# Example data structure with titles
df = pd.DataFrame({
    'recipe_id': [10, 20, 30, 40],
    'title': ['Chocolate Cake', 'Chicken Salad', 'Beef Stew', 'Garlic Shrimp'],
    'ingredients': [
        'flour, sugar, cocoa powder, baking powder, baking soda, kosher salt, espresso powder, milk, oil, egg, vanilla extract, water, vanilla buttercream frosting',
        'chicken, salt, pepper, olive oil, garlic, onion, tomatoes',
        'beef, onion, garlic, soy sauce, sugar, vinegar, water, bay leaves',
        'shrimp, garlic, butter, parsley, lemon, salt, pepper'
    ]
})

# Convert DataFrame to TensorFlow Dataset
tf_dataset = tf.data.Dataset.from_tensor_slices({
    'recipe_id': df['recipe_id'].values,
    'title': df['title'].values,
    'ingredients': df['ingredients'].values
})

# Shuffle and batch the dataset
tf_dataset = tf_dataset.shuffle(10000).batch(128).cache()

class RecipeModel(tfrs.Model):

    def __init__(self, ingredient_vocab, embedding_dimension=32):
        super().__init__()

        # Define recipe embedding layers
        self.ingredient_lookup = tf.keras.layers.StringLookup(vocabulary=ingredient_vocab, mask_token=None)
        self.ingredient_embedding = tf.keras.layers.Embedding(len(ingredient_vocab) + 1, embedding_dimension)

        # Define the task
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=tf.data.Dataset.from_tensor_slices(ingredient_vocab).batch(128).map(self.compute_candidate_embeddings)
            )
        )

    def compute_candidate_embeddings(self, ingredients):
        ingredients_split = tf.strings.split(ingredients, sep=', ')
        ingredient_ids = self.ingredient_lookup(ingredients_split)
        ingredient_embeddings = self.ingredient_embedding(ingredient_ids)
        recipe_embeddings = tf.reduce_mean(ingredient_embeddings, axis=1)
        return recipe_embeddings

    def compute_query_embeddings(self, ingredients):
        ingredients_split = tf.strings.split(ingredients, sep=', ')
        ingredient_ids = self.ingredient_lookup(ingredients_split)
        ingredient_embeddings = self.ingredient_embedding(ingredient_ids)
        recipe_embeddings = tf.reduce_mean(ingredient_embeddings, axis=1)
        return recipe_embeddings

    def call(self, features):
        # Forward pass to compute embeddings
        if isinstance(features, dict):
            return self.compute_query_embeddings(features['ingredients'])
        else:
            return self.compute_query_embeddings(features)

    def compute_loss(self, features, training=False):
        query_embeddings = self(features)
        return self.task(query_embeddings=query_embeddings, candidate_embeddings=query_embeddings)

# Create vocabulary of ingredients
ingredient_vocab = sorted(set(', '.join(df['ingredients']).split(', ')))

# Instantiate the model
model = RecipeModel(ingredient_vocab)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Train the model
model.fit(tf_dataset, epochs=5)

# Save the model configuration and weights separately
model_config = model.get_config()
model_weights = model.get_weights()

# Save the configuration and weights to disk
with open('recipe_model_config.pkl', 'wb') as f:
    pickle.dump(model_config, f)
with open('recipe_model_weights.pkl', 'wb') as f:
    pickle.dump(model_weights, f)

# Load the configuration and weights from disk
with open('recipe_model_config.pkl', 'rb') as f:
    loaded_config = pickle.load(f)
with open('recipe_model_weights.pkl', 'rb') as f:
    loaded_weights = pickle.load(f)

# Reconstruct the model
reconstructed_model = RecipeModel(ingredient_vocab)
sample_input = {'ingredients': tf.constant(["salt, pepper"])}
reconstructed_model(sample_input)
reconstructed_model.set_weights(loaded_weights)
reconstructed_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Use BruteForce layer to find top recommendations
index = tfrs.layers.factorized_top_k.BruteForce(reconstructed_model)

# Prepare candidates dataset
candidate_embeddings = tf.data.Dataset.from_tensor_slices({
    'recipe_id': df['recipe_id'].values,
    'title': df['title'].values,
    'ingredients': df['ingredients'].values
}).batch(128).map(lambda x: (x['recipe_id'], reconstructed_model.compute_candidate_embeddings(x['ingredients'])))

# Set the index from the candidate embeddings
index.index_from_dataset(candidate_embeddings)

# Get recommendations
example_recipe = np.array(["salt, pepper"])
example_recipe_tensor = tf.constant(example_recipe)
scores, top_k = index(example_recipe_tensor, k=4)  # Set k explicitly

print("Top recommendations:")
for i in range(len(top_k[0])):
    recipe_id = top_k[0][i].numpy()
    title = df[df['recipe_id'] == recipe_id].title.values[0]
    ingredients = df[df['recipe_id'] == recipe_id].ingredients.values[0]
    print(f"Title: {title}\nIngredients: {ingredients}\n")





Epoch 1/5
1/1 [==============================] - 2s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.5000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 5.5353 - regularization_loss: 0.0000e+00 - total_loss: 5.5353
Epoch 2/5
1/1 [==============================] - 0s 97ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.5000 - factorized_top_k/top_10_categorical_accuracy: 1.0000 - factorized_top_k/top_50_categorical_accuracy: 1.0000 - factorized_top_k/top_100_categorical_accuracy: 1.0000 - loss: 5.5339 - regularization_loss: 0.0000e+00 - total_loss: 5.5339
Epoch 3/5
1/1 [==============================] - 0s 84ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.5000 - fac

In [1]:
%pip install tensorflow==2.15.0

  Using cached cachetools-5.3.3-py3-none-any.whl.metadata (5.3 kB)
   ---------------------------------------- 0.0/300.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/300.8 MB 1.6 MB/s eta 0:03:08
   ---------------------------------------- 0.3/300.8 MB 3.5 MB/s eta 0:01:26
   ---------------------------------------- 0.6/300.8 MB 5.0 MB/s eta 0:01:01
   ---------------------------------------- 1.1/300.8 MB 6.1 MB/s eta 0:00:49
   ---------------------------------------- 1.5/300.8 MB 6.8 MB/s eta 0:00:44
   ---------------------------------------- 1.6/300.8 MB 6.8 MB/s eta 0:00:45
   ---------------------------------------- 1.8/300.8 MB 5.7 MB/s eta 0:00:53
   ---------------------------------------- 2.8/300.8 MB 7.8 MB/s eta 0:00:39
   ---------------------------------------- 3.3/300.8 MB 8.0 MB/s eta 0:00:38
   ---------------------------------------- 3.7/300.8 MB 8.1 MB/s eta 0:00:37
    --------------------------------------- 3.8/300.8 MB 8.3 MB/s eta 0:00:36
    

In [22]:
import sys
print("Python version:", sys.version)


Python version: 3.9.13 (tags/v3.9.13:6de2ca5, May 17 2022, 16:36:42) [MSC v.1929 64 bit (AMD64)]
